# [Unilever Indonesia] OTE Project Solana (COps Data Tech Revamp) - Technical Test Section
# Abdurrahman Nur Ashri Diasta Fajar Ramadlan

In [35]:
# Import libraries
import pandas as pd
import numpy as np
import datetime

import warnings
warnings.filterwarnings('ignore')

In [36]:
# Load input data
FBL5N=pd.read_excel("CAOT Test Case _ Final.xlsx",sheet_name="Input Data - FBL5N")
cushier=pd.read_excel("CAOT Test Case _ Final.xlsx",sheet_name="Input Data - Cust Hierarchy")
holiday=pd.read_excel("CAOT Test Case _ Final.xlsx",sheet_name="Input - Holiday")
reportdate=pd.read_excel("CAOT Test Case _ Final.xlsx",sheet_name="Input - Report Date")

In [37]:
# Load output data 
out2=pd.read_excel("CAOT Test Case _ Final.xlsx",sheet_name="Output",skiprows=[0,1,2,3,4])

In [38]:
# Merge data to generate the first 5 processed fields
out=pd.merge(FBL5N.iloc[:,:2],cushier.iloc[:,[0,1,3,13,10,11,2]],how="left",left_on="Account",right_on="Customer")
out=out.drop(columns="Customer")
out

,Company Code,Account,Customer Name,TFL2,Customer Channel,SWL2,SWL3,Banner
0,2408,15061017,PE. SXGKCA QLCQAWQ EAWJQYQ YC KQIYX,GZYCAI EAQYC,MT,CYG GWIWGQAKCE,KQG QLCQGQAE,QLCQGQAE
1,2408,15061063,PE. EXLXS GQJX,YWSEAWKXEWVC EAQYC,UFS,AKG CCIEAQL,QKG CQSECAI JQKQAEQ,PE. EXLXS GQJX
2,2408,15061820,CLQIYYS GQAE,GZYCAI EAQYC,MT,ASG ZXECA WSLQIY,KXH KQLW,CLQIYYS GQAE
3,2408,15061820,CLQIYYS GQAE,GZYCAI EAQYC,MT,ASG ZXECA WSLQIY,KXH KQLW,CLQIYYS GQAE
4,2408,15062513,CV. XSQHQ JQYQ QKQYW,YWSEAWKXEWVC EAQYC,DT,ASG ZXECA WSLQIY,QSG PZIEWQIQK,XSQHQ JQYQ
...,...,...,...,...,...,...,...,...
2808,2408,15553734,JXAQOQI KZOZA CWPQIQS,YWSEAWKXEWVC EAQYC,IC,ASG SCAX,QSG SCAX 2,JXAQOQI KZOZA CWPQIQS
2809,2408,15553734,JXAQOQI KZOZA CWPQIQS,YWSEAWKXEWVC EAQYC,IC,ASG SCAX,QSG SCAX 2,JXAQOQI KZOZA CWPQIQS
2810,2408,15612480,JXAQOQI KQAQWQIO CWKXQYQ,YWSEAWKXEWVC EAQYC,IC,ASG SCAX,QSG SCAX 2,JXAQOQI KQAQWQIO CWKXQYQ
2811,2408,15612948,JXAQOQI KZOZA CWSQAXQ,YWSEAWKXEWVC EAQYC,IC,ASG SCAX,QSG SCAX 2,JXAQOQI KZOZA CWSQAXQ


In [39]:
# Missing value check
out.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2813 entries, 0 to 2812
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Company Code      2813 non-null   int64 
 1   Account           2813 non-null   int64 
 2   Customer Name     2813 non-null   object
 3   TFL2              2812 non-null   object
 4   Customer Channel  2813 non-null   object
 5   SWL2              2812 non-null   object
 6   SWL3              2812 non-null   object
 7   Banner            2812 non-null   object
dtypes: int64(2), object(6)
memory usage: 197.8+ KB


In [40]:
# Handling missing value
for x in out.columns[2:]:
    out[x]=out[x].fillna("Not Found")

In [41]:
# Re-checking missing value
out.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2813 entries, 0 to 2812
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Company Code      2813 non-null   int64 
 1   Account           2813 non-null   int64 
 2   Customer Name     2813 non-null   object
 3   TFL2              2813 non-null   object
 4   Customer Channel  2813 non-null   object
 5   SWL2              2813 non-null   object
 6   SWL3              2813 non-null   object
 7   Banner            2813 non-null   object
dtypes: int64(2), object(6)
memory usage: 197.8+ KB


In [42]:
# Displaying handled missing value
out[out['SWL2']=="Not Found"]

,Company Code,Account,Customer Name,TFL2,Customer Channel,SWL2,SWL3,Banner
971,2408,15080950,PE IXSQ PAWGQ PQIOQI,Not Found,Others,Not Found,Not Found,Not Found


In [43]:
# Adding unprocessed fields to output
out=out.join(FBL5N.iloc[:,2:])

In [44]:
# Defining function for `Exclude` field
def excl(row):
    refkey=row["Reference Key 1"]
    if "CITDUMMY" in str(refkey) or "CITDUMMYUFS" in str(refkey) or "CITDUMMY UFS" in str(refkey) or "CITDUMMY-NKW" in str(refkey):
        return row["Reference Key 1"]
    elif row["Amount in local currency"] > 0:
        return "Positive Value"
    elif row["Local Currency"]!="IDR" or row["Document currency"]!="IDR":
        return "Not IDR"
    elif str(row["Document Number"]).startswith('58'):
        return "Reversal Document"
    elif "bnppay" in str(row["Reference"]) or "bnppay" in str(row["Text"]):
        return "BNPPAY"
    else:
        return None

In [45]:
# Applying excl function
out["Exclude"]=out.apply(excl,axis=1)

In [46]:
# Defining function for `Days` field
holidays=holiday["Holiday"].apply(lambda x:x.date()).tolist()
daylist=[]

for i in range(len(out)):
    if pd.isna(out["Clearing Document"][i]):
        end=reportdate.columns[0].date()
        count=np.busday_count(out["Document Date"][i].date(),end,holidays=holidays)
        if count<=0:
            daylist.append(0)
        else:
            daylist.append(count)
    else:
        end=out["Clearing date"][i].date()
        count=np.busday_count(out["Document Date"][i].date(),end,holidays=holidays)
        if count<=0:
            daylist.append(0)
        else:
            if (end.weekday()>4) or (end in holidays):
                daylist.append(count-1)
            else:
                daylist.append(count)

# Applying function
out["Days"]=daylist

In [47]:
# Defining function for `Ageing` field
def aging(a):
    if a["Days"] <= -1:
        return ">2"
    elif a["Days"] < 2:
        return "<2"
    elif a["Days"] == 2:
        return "=2"
    else:
        return ">2"

# Applying the function
out["Ageing"]=out.apply(aging,axis=1)

In [48]:
# Generating `Clearing Status` field
out["Clearing Status"]=out["Clearing Document"].apply(lambda s:"Open" if pd.isna(s) else "Cleared")

# Group by Output

In [49]:
# Group by Exclude- Count of rows
display(out["Exclude"].value_counts(dropna=False).sort_index())
print("Grand Total : ",out["Exclude"].value_counts(dropna=False).sort_index().sum())

BNPPAY                  2
CITDUMMY              176
CITDUMMY UFS            2
CITDUMMY-NKW            7
CITDUMMYUFS           105
Not IDR                 8
Positive Value       1278
Reversal Document       2
None                 1233
Name: Exclude, dtype: int64

Grand Total :  2813


In [50]:
# Group by Channel- Count of days with <=2 days and exclude field is empty
display(out[(out["Exclude"].isna())&((out["Ageing"]=="=2")|(out["Ageing"]=="<2"))]["Customer Channel"].value_counts())
print("Grand Total : ",out[(out["Exclude"].isna())&((out["Ageing"]=="=2")|(out["Ageing"]=="<2"))]["Customer Channel"].value_counts().sum())

DT        536
MT        324
IC        277
UFS        41
U-PRO       9
Others      1
Name: Customer Channel, dtype: int64

Grand Total :  1188


# Excel Output

In [51]:
# Save output as excel file
out.to_excel(r"Technical Test Submission excel - Abdurrahman Nur Ashri Diasta Fajar Ramadlan.xlsx",index=False)